In [1]:
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import datetime
import os

import dateutil.parser as dparser

import requests
import pandas as pd
from bs4 import BeautifulSoup

import cPickle

In [3]:
import seaborn as sns
sns.set_style('darkgrid')

In [4]:
pd.__version__

u'0.18.1'

In [5]:
def pegar_links():

    lista_links = []
    for p in range(10):
        pagina = p * 100
        
        url = "http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/?b_start:int={0}".format(pagina)
        
        r = requests.get(url)
        content = BeautifulSoup(r.content)
        links = content.find_all("a", {'class': 'summary url'})
        links = [l['href'] for l in links]
        
        lista_links.extend(links)
        
    return lista_links

In [6]:
lista_links = pegar_links()

//anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [7]:
if not os.path.exists('lista_links.pkl'):
    print "Arquivo com links não encontrado, baixando links"
    lista_links  = pegar_links()
    
    f = open('lista_links.pkl', 'wb')
    cPickle.dump(lista_links, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()
else:
    print "Lendo arquivos de links do disco"
    f = open('lista_links.pkl', 'rb')
    lista_links = cPickle.load(f)
    f.close()
    

Lendo arquivos de links do disco


In [8]:
def baixar_discurso(url):
    try:
        r = requests.get(url)

        content = BeautifulSoup(r.content)

        titulo = content.find("h1", {"class": "documentFirstHeading"}).text
        description = content.find("div", {"class": "description"})
        if description != None:
            description = description.text
        document_published = (content.find("span", {"class": "documentPublished"}).text
                              .strip().replace("publicado", "").strip().replace(",", ""))
        document_published = datetime.datetime.strptime(document_published, "%d/%m/%Y %Hh%M")

        document_modified = content.find("span", {"class": "documentModified"})
        if document_modified != None:
            document_modified = (document_modified.text.strip()
                                 .replace(u"última modificação", "").strip().replace(",", ""))

            document_modified = datetime.datetime.strptime(document_modified, "%d/%m/%Y %Hh%M")

        texto = content.find("div", {"id": "parent-fieldname-text"}).text

        tags = content.find("div", {"id": "category"})
        if tags != None:
            tags = [t.text for t in tags.find_all("a")]
        else:
            tags = []

        assuntos = content.find("div", {"id": "category-vcge"})
        if assuntos != None:
            assuntos = [a.text for a in assuntos.find_all("a")]
        else:
            assuntos = []
        
        d = {
            'titulo': titulo,
            'descricao': description,
            'data_publicacao': document_published,
            'data_modificacao': document_modified,
            'texto': texto,
            'tags': tags,
            'assuntos': assuntos,
            'url': url
        }
        return d
    
    except Exception as e:
        print "Erro ao baixar a url: {0}".format(url)
        print e
        
        return None

In [9]:

if not os.path.exists('discursos.pkl'):
    print "Arquivo com discursos não encontrado, baixando discursos"

    discursos = []
    for url in lista_links:
        discurso = baixar_discurso(url)
        if discurso != None:
            discursos.append(discurso)

    f = open('discursos.pkl', 'wb')
    cPickle.dump(discursos, f, protocol=cPickle.HIGHEST_PROTOCOL)
    f.close()
else:
    print "Lendo arquivos de discursos do disco"
    f = open('discursos.pkl', 'rb')
    discursos = cPickle.load(f)
    f.close()
    

Lendo arquivos de discursos do disco


In [10]:
"""
Erro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/mensagem-ao-congresso-2012/view
'NoneType' object has no attribute 'text'
Erro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/speech-by-the-president-of-the-republic-dilma-rousseff-brazilian-national-congress-january-1st-2015
time data ':\n    06/01/2015 11h31' does not match format '%d/%m/%Y %Hh%M'
Erro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/discurso-de-la-presidenta-de-la-republica-dilma-rousseff-durante-el-compromiso-constitucional-ante-el-congreso-nacional-congreso-nacional-1deg-de-enero-de-2014
time data ':\n    06/01/2015 17h29' does not match format '%d/%m/%Y %Hh%M'
Erro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/address-by-the-president-of-the-federative-republic-of-brazil-during-the-3rd-celac-summit
time data ':\n    11/02/2015 18h35' does not match format '%d/%m/%Y %Hh%M'
Erro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/discurso-de-la-presidenta-de-la-republica-en-la-iii-cumbre-de-la-celac
time data ':\n    11/02/2015 18h38' does not match format '%d/%m/%Y %Hh%M'
"""

"\nErro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/mensagem-ao-congresso-2012/view\n'NoneType' object has no attribute 'text'\nErro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/speech-by-the-president-of-the-republic-dilma-rousseff-brazilian-national-congress-january-1st-2015\ntime data ':\n    06/01/2015 11h31' does not match format '%d/%m/%Y %Hh%M'\nErro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/discurso-de-la-presidenta-de-la-republica-dilma-rousseff-durante-el-compromiso-constitucional-ante-el-congreso-nacional-congreso-nacional-1deg-de-enero-de-2014\ntime data ':\n    06/01/2015 17h29' does not match format '%d/%m/%Y %Hh%M'\nErro ao baixar a url: http://www2.planalto.gov.br/acompanhe-o-planalto/discursos/discursos-da-presidenta/address-by-the-president-of-the-federative-republic-of-brazil-during-the-3rd-celac-summ

In [11]:
df = pd.DataFrame().from_records(discursos)

In [12]:
df = df.sort_values(by='data_publicacao')

In [13]:
df = df.set_index('data_publicacao')

In [14]:
def pegar_duracao(t):
    tf = t[-60:]
    prt_ini = tf.find("(")
    filtro_duracao = tf[tf.find("(")+1:tf.find(")")].strip()
    
    if "discurso" in tf and prt_ini != -1:
        filtro_duracao = filtro_duracao.replace("min", "m")
        if filtro_duracao == "60m01s":
            filtro_duracao = "1h1s"
        if filtro_duracao.strip() == "":
            return None
        return dparser.parse(filtro_duracao, fuzzy=True).time()
    else:
        return None

In [15]:
df['duracao'] = df.texto.apply(lambda x: pegar_duracao(x))

In [16]:
def pegar_ocorrencia(col, s):
    return col.str.count(s)